### Count how many comparisons developers have made

In [ ]:
import database

THRESHOLD = 10
PROJECT='torvalds-linux'

num_participants = 0
num_answers = 0
for c in database.stats_compared(PROJECT):
    if c[1] >= THRESHOLD:
        num_participants += 1
        num_answers += c[1]
        print(c[0], c[1])
print('# participants: %d\t# answers: %d' % (num_participants, num_answers))

### List compared pairs along with the reasons

#### For individual projects

In [ ]:
import database

PROJECT='torvalds-linux'
LINK='https://github.com/torvalds/linux/commit/'

comparisons = [c for c in database.list_project_compared(PROJECT)]
comparisons = sorted(comparisons, key=lambda x: x['commit1'])
for c in comparisons:
    print(LINK + c['commit1'], LINK + c['commit2'], c['reason'], sep='\t')

#### For all projects

In [ ]:
from common import github_commit_url
import database

comparisons = [c for c in database.list_compared()]
comparisons = sorted(comparisons, key=lambda x: x['commit1'])

n_self = 0
n_other = 0
with open('js_all_comparisons.csv', 'w') as f:
    for c in comparisons:
        if c['author'] == c['email']:
            n_self += 1
        else:
            n_other += 1
        url1 = github_commit_url(c['project'], c['commit1'], True)
        url2 = github_commit_url(c['project'], c['commit2'], True)
        print(c['project'], c['author'], url1, c['title1'], c['email'],
              url2, c['title2'], c['reason'], sep='\t', file=f)
        
    print('# self comparisons:\t%d' % n_self)
    print('# other comparisons:\t%d' % n_other)

### List individual commits and their reviews

#### For individual projects

In [ ]:
import common
import database

PROJECT='torvalds-linux'

commit2desc = dict()
commit2title = dict()


def add_to_dict(commit, title, desc):
    if commit not in commit2desc:
        commit2desc[commit] = [desc]
    elif desc not in commit2desc[commit]:
        commit2desc[commit].append(desc)
    
    if commit not in commit2title:
        commit2title[commit] = title
    else:
        assert commit2title[commit] == title


for c in database.list_compared():
    d1, d2 = common.parse_descriptions(c['reason'])
    add_to_dict(c['commit1'], c['title1'], '(+) ' + d1)
    add_to_dict(c['commit2'], c['title2'], '(-) ' + d2)
    
for i in commit2desc.items():
    print(i[0], commit2title[i[0]], i[1], sep='\t')

#### For all projects

In [ ]:
import common
import database

url2info = dict()


def add_to_dict(url, title, desc, self: bool):
    if url not in url2info:
        url2info[url] = dict()
    info = url2info[url]
    
    if 'title' not in info:
        info['title'] = title
    else:
        assert info['title'] == title
    
    if 'self' not in info:
        info['self'] = []
    if 'other' not in info:
        info['other'] = []
    
    if self:
        for d in info['self']:
            if d[4:] == desc[4:]:
                desc = d[:4] + desc
                info['self'].remove(d)
                info['self'].append(desc)
                return
        info['self'].append(desc)
    else:
        for d in info['other']:
            if d[4:] == desc[4:]:
                desc = d[:4] + desc
                info['other'].remove(d)
                info['other'].append(desc)
                return
        info['other'].append(desc)


for c in database.list_compared():
    d1, d2 = common.parse_descriptions(c['reason'])
    url1 = common.github_commit_url(c['project'], c['commit1'], True)
    url2 = common.github_commit_url(c['project'], c['commit2'], True)
    add_to_dict(url1, c['title1'], '(+) ' + d1, c['author'] == c['email'])
    add_to_dict(url2, c['title2'], '(-) ' + d2, c['author'] == c['email'])

with open('js_commit_list.csv', 'w') as f:
    for url, info in url2info.items():
        print(url, info['title'], info['self'], info['other'], sep='\t', file=f)